### Libraries and datasets required

original dataset from  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
Images: 3000 (2000 Training, 1000 Validation) 

In [96]:
import os
import pandas as pd
import tfrecorder
import wandb 
print(wandb.__version__)
import tensorflow as tf
import time

0.10.4


In [97]:
os.system('wandb login 3a6710e811d34207ea03768ba12e7ea6c8a9fefd')
os.environ['WANDB_NOTEBOOK_NAME'] = 'data_prep.ipynb'

### GCS Fuse to be able to use os utilities on GCS without copying data

In [98]:
#!gcsfuse --implicit-dirs mchrestkha-demo-env-ml-examples /home/jupyter/gcs/ 
#!fusermount -u /home/jupyter/gcs/

### Collect all image URIs

In [108]:
RUN_NAME=time.strftime("set_up_data_%Y%m%d_%H%M%S")
run = wandb.init(project='cats-dogs-keras', job_type='data', name=RUN_NAME)

In [115]:
wandb.log({"cat_examples": wandb.Image("/home/jupyter/gcs/catsdogs/train/cats/cat.1.jpg", caption="Cat1")})
wandb.log({"cat_examples": wandb.Image("/home/jupyter/gcs/catsdogs/train/cats/cat.2.jpg", caption="Cat2")})
wandb.log({"dog_examples": wandb.Image("/home/jupyter/gcs/catsdogs/train/dogs/dog.1.jpg", caption="Dog1")})
wandb.log({"dog_examples": wandb.Image("/home/jupyter/gcs/catsdogs/train/dogs/dog.2.jpg", caption="Dog2")})

In [ ]:
myDir='/home/jupyter/gcs/catsdogs/'
format='.jpg'
fileList = []
for root, dirs, files in os.walk(myDir, topdown=False):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
                
fileList[:10]

In [ ]:
df=pd.DataFrame(fileList)
df.columns = ['image_uri']
df.head()

In [ ]:
df.loc[df['image_uri'].str.contains('train'), 'split'] = 'TRAIN'
df.loc[df['image_uri'].str.contains('validation'), 'split'] = 'VALIDATION'
df.loc[df['image_uri'].str.contains('|'.join(['train/cats', 'validation/cats'])), 'label'] = 'cats'
df.loc[df['image_uri'].str.contains('|'.join(['train/dogs', 'validation/dogs'])), 'label'] = 'dogs'
df = df[['split', 'image_uri', 'label']]
df.head()

In [ ]:
dflocal=df.copy()
dfgcs=df.copy()
dfgcs['image_uri'] = dfgcs['image_uri'].str.replace('/home/jupyter/gcs','gs://mchrestkha-demo-env-ml-examples')
dflocal.to_csv('../data/datalocal1.csv', index=False)
dfgcs.to_csv('../data/datagcs1.csv', index=False)

In [ ]:
artifact = wandb.Artifact(name='training_images', type='dataset')
artifact.add_reference('gs://mchrestkha-demo-env-ml-examples/catsdogs/train/')
run.log_artifact(artifact)
artifact = wandb.Artifact(name='validation_images', type='dataset')
artifact.add_reference('gs://mchrestkha-demo-env-ml-examples/catsdogs/validation/')
run.log_artifact(artifact)
artifact = wandb.Artifact(name='image_uris_csv', type='dataset')
artifact.add_file('../data/datagcs.csv')
run.log_artifact(artifact)
run.finish()

In [ ]:
RUN_NAME=time.strftime("generate_tfrecords_%Y%m%d_%H%M%S")
run = wandb.init(project='cats-dogs-keras',job_type='data', name=RUN_NAME)
artifact = run.use_artifact('training_images:latest')
artifact = run.use_artifact('validation_images:latest')
artifact = run.use_artifact('image_uris_csv:latest')

In [ ]:
FILENAME='../data/datagcs.csv'
TFRECORD_OUTPUT='gs://mchrestkha-demo-env-ml-examples/catsdogs/tfrecords'
PROJECT='mchrestkha-demo-env'
REGION='us-west1'
TFRECORDER_WHEEL='/home/jupyter/tfrecorder_wheel/tfrecorder-0.1.2-py3-none-any.whl'


dfgcs = pd.read_csv(FILENAME)
dfgcs.tensorflow.to_tfr(
    output_dir=TFRECORD_OUTPUT,
    runner='DataflowRunner',
    project=PROJECT,
    region=REGION,
    tfrecorder_wheel=TFRECORDER_WHEEL)

In [ ]:
dfgcs

In [ ]:
artifact = wandb.Artifact(name='tfrecords', type='dataset')
artifact.add_reference('gs://mchrestkha-demo-env-ml-examples/catsdogs/tfrecords/')
run.log_artifact(artifact)
run.finish()